# Kelompok 7 - Firefly Algorithm
Ketua Kelompok : 120450009 - Alfian Dwi Kurnia

Anggota 1 : 120450015 - Alfa Khoirin

Anggota 2 : 120450081 - Dimas Wahyu Saputro

Anggota 3 : 120450069 - Rezki P. Manullang


# Pengolahan Data ke CSV

Dapat diakses [disini](https://iteraacid-my.sharepoint.com/:f:/g/personal/dimas_120450081_student_itera_ac_id/EqqHYRXM2X5Hn5o1VCQpxdYB8g56gv-CGH1B70r89Xr1iQ?e=uGvAOC)

# Pengkodean FA

In [23]:
import numpy as np
import pandas as pd


df = pd.read_csv('dataFinal.csv')
df.set_index('Nama Gedung', inplace=True)
df.head()

,Asrama 1,Asrama 2,Asrama 3,Asrama 4,At Tanwir Mosque,Baitul Ilmi Mosque,Galeri,Gedung A ITERA,Gedung B ITERA,Gedung C ITERA,...,Gedung Labtek 2 ITERA,Gedung Labtek 3 ITERA,Gerbang Utama,GKU 1,GSG Itera,Wisma,Kantin BKL ITERA,Kantin Rumah Kayu,Poliklinik ITERA,Rumah Ibadah Multi Agama
Nama Gedung,,,,,,,,,,,,,,,,,,,,,
Asrama 1,0.000000,0.212562,0.179391,0.018479,0.168656,0.544826,0.213063,0.335797,0.246667,0.401103,...,0.756409,0.917457,0.291970,0.792995,0.201048,0.175924,0.107276,0.541684,0.169328,0.432174
Asrama 2,0.212562,0.000000,0.039762,0.205152,0.075938,0.757345,0.425110,0.548052,0.458784,0.613644,...,0.968041,1.129832,0.503702,1.005080,0.134634,0.378350,0.317139,0.754156,0.380709,0.219652
Asrama 3,0.179391,0.039762,0.000000,0.170212,0.076850,0.722972,0.392444,0.515077,0.426027,0.579937,...,0.932355,1.094956,0.471360,0.969811,0.105122,0.349962,0.285774,0.719629,0.348627,0.254268
Asrama 4,0.018479,0.205152,0.170212,0.000000,0.167377,0.552772,0.223295,0.345375,0.256673,0.409808,...,0.762890,0.924915,0.302326,0.799989,0.184647,0.190879,0.120367,0.549454,0.180567,0.424184
At Tanwir Mosque,0.168656,0.075938,0.076850,0.167377,0.000000,0.707505,0.373170,0.495626,0.406651,0.562672,...,0.921649,1.080464,0.450103,0.957203,0.177849,0.318148,0.264071,0.704782,0.328011,0.279118


In [25]:
# Inisialisasi Individu-Populasi & Diskritisasi (Ranking untuk TSP / Sigmoid untuk KP)

gen_individu = lambda n_individu, n_kota, a, b: np.random.uniform( a,b,(n_individu, n_kota) )

def calc_dist(X, adj_mat):
    return sum( map( lambda x, y: adj_mat[x,y], X, np.roll(X,-1) ) )

def diskritisasi(kunangs):
    return np.argsort(kunangs)

def calculate_fitness(kunangs, df):
    d_kunangs = diskritisasi(kunangs)
    fitness = np.array( list(map( lambda x: calc_dist( x, df.values), d_kunangs )) )
    fitness = fitness.reshape( (-1,1) )
    return np.concatenate( (kunangs, fitness), axis=1 )

def sort_individu(kunangs_with_f):
    return kunangs_with_f[ kunangs_with_f[:,-1].argsort() ]

def solusi(kunangs_w_f):
    df_kota = pd.DataFrame(diskritisasi(kunangs_w_f[:,:-1]))
    cols = ['Urutan ' + str(i+1) for i in range( df_kota.shape[1]) ]
    df_kota.columns = cols
    df_kota['Jarak'] = kunangs_w_f[:,-1].reshape(-1,1)
    return df_kota

def inisialisasi(params, df):
    kunangs = gen_individu(int(params['n_individu']), int(params['n_kota']), params['a'], params['b'])
    kunangs_w_f = sort_individu(calculate_fitness(kunangs, df))
    return kunangs_w_f

def movement( X, i, j, p ):
    r = np.linalg.norm(X[i,:] - X[j,:])
    term1 = p['beta0'] * np.exp(-1 * p['gamma'] * r**2 )
    term2 = X[j,:] - X[i,:]
    term3 = p['alpha'] * np.random.uniform(0,1)
    return X[i,:] + ( term1 * term2 ) + term3

def FA(params, df):

    generasi = 0
    new_kunangs_w_f = inisialisasi(params, df)
    temp = np.zeros_like(new_kunangs_w_f[:,:-1])

    while generasi < params['max_generasi']:

        for i in range(int(params['n_individu'])):
            for j in range(int(params['n_individu'])):
                if (new_kunangs_w_f[i,-1] > new_kunangs_w_f[j,-1] ) and (i != j):
                    temp[i,:] = movement(new_kunangs_w_f[:,:-1], i, j, params)
                    
        new_kunangs_w_f = sort_individu(calculate_fitness(temp, df))
        temp = new_kunangs_w_f[:,:-1]

        generasi += 1

    return solusi(new_kunangs_w_f)

def run_FA(dfparams, df):
    return [ FA( dfparams.loc[i].to_dict(), df) for i in range(dfparams.shape[0]) ]

# jangan lupa ganti nama csv, supaya mempermudah. Sesuaikan dengan dfparams.
def save_FA(hasils):
    for h in enumerate(hasils):
        pd.DataFrame(h[1]).to_csv('hasil_FA' + str(h[0]) + '.csv')

In [26]:
# Inisialisasi Parameter pada metode

# Main program
# sesuaikan dfparams dengan berbagai params yang sudah dibuat
dfparams = pd.read_csv('/content/FA_params.csv')
dfparams['n_kota'] = 24

In [31]:
# Pengkodean tahap kedua - metode masing masing kelompok 
hasils = run_FA(dfparams, df)

In [32]:
save_FA(hasils)

# Analisis

Disesuaikan dengan soal yang ada, hasil csv dari pengkodean, maupun hasil visualisasi, dapat diakses [disini](https://iteraacid-my.sharepoint.com/personal/dimas_120450081_student_itera_ac_id/_layouts/15/onedrive.aspx?ga=1&id=%2Fpersonal%2Fdimas%5F120450081%5Fstudent%5Fitera%5Fac%5Fid%2FDocuments%2FSemester%204%2FSwarm%20Intelligence%2FKelompok%2FProgress%20Final)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
for i in range(0,8):

    # sesuaikan dengan csv yang didapatkan dari Pengkodean
    # kenapa i? supaya bisa berulang secara otomatis.
    df_open = pd.read_csv(f'/content/hasil_FA_x{i}.csv')
    df = df_open.drop(df_open.iloc[:, [0]], axis=1)

    df1 = pd.DataFrame(df['Jarak'])
    df1['Firefly'] = range(df1.shape[0])

    plt.figure(figsize=(5,5))
    ax = plt.gca()

    # jangan lupa ubah title plot
    ax.set_title(f'Solusi TSP dalam populasi FA - final - {i+1}', fontsize = 15)
    ax.set_ylabel('Jarak (m) ', fontsize = 10)
    ax.set_xlabel('Individu Firefly', fontsize = 10)
    ax.set_yticks(np.arange(1,15,0.25))
    df1.plot(ax=ax, kind='scatter', y = 'Jarak', x = 'Firefly')

    # Untuk menyimpan plot
    # plt.savefig(f'final{i+1}.png')

Tambahan:
[Analisis percobaan dan error.](https://iteraacid-my.sharepoint.com/:u:/g/personal/dimas_120450081_student_itera_ac_id/EfqHUMvQoZBNl2q7JYsE3P0BVngTPHuRJMXZxoOEMc5hPw?e=EIk18h)

# Additional: TSP GA dan Generate a Plot

Dapat diakses [disini](https://iteraacid-my.sharepoint.com/:u:/g/personal/dimas_120450081_student_itera_ac_id/EWhuiM048SVDvNLZDmYMwAwB9XgAbllBK61gGG1Ij4yMiw?e=OAd3Ar)

# Additional: Bahan Belajar

In [ ]:
gen_individu = lambda n_individu, n_kota, a, b: np.random.uniform( a,b,(n_individu, n_kota) )

In [ ]:
def inisialisasi(params, df):
    kunangs = gen_individu(int(params['n_individu']), int(params['n_kota']), params['a'], params['b'])
    kunangs_w_f = sort_individu(calculate_fitness(kunangs, df))
    return kunangs_w_f

In [ ]:
params = {
    'n_individu':4,
    'a':1,
    'b':10,
    'n_kota': 24
}

In [ ]:
kunangs = gen_individu(int(params['n_individu']), int(params['n_kota']), params['a'], params['b'])
kunangs

array([[7.91348267, 1.49501169, 2.59715461, 4.01687221, 8.15752291,
        4.24880893, 1.02554559, 8.03379343, 8.83240293, 5.42219789,
        9.91206883, 3.16255079, 9.65081878, 1.49283316, 5.29332248,
        8.54417041, 8.54662427, 8.35472003, 9.51850189, 2.59406872,
        6.3894718 , 3.42192787, 5.76101831, 7.72938651],
       [4.63501508, 4.97860023, 7.5483839 , 4.46017026, 8.76932727,
        4.04407313, 8.51815035, 5.33955442, 3.46262213, 7.08256716,
        6.6121777 , 7.14527499, 4.75870229, 8.13551065, 6.78056696,
        8.67635263, 2.64165705, 8.93148302, 5.01506629, 5.95743055,
        4.23073229, 9.50899547, 9.92359357, 9.10162877],
       [5.34812567, 9.76793648, 6.8310175 , 8.53917504, 7.63431022,
        9.76316366, 1.956988  , 4.2742454 , 6.14971457, 1.63646774,
        7.45632586, 1.2629425 , 7.18041886, 9.64337859, 5.14809428,
        2.24573506, 2.92470499, 1.75134764, 9.46049487, 3.97718096,
        9.25040331, 3.20078346, 9.9881189 , 6.55069277],
       [9.878

In [ ]:
def calculate_fitness(kunangs, df):
    d_kunangs = diskritisasi(kunangs)
    fitness = np.array( list(map( lambda x: calc_dist( x, df.values), d_kunangs )) )
    print(fitness)
    fitness = fitness.reshape( (-1,1) )
    return np.concatenate( (kunangs, fitness), axis=1 )

In [ ]:
calculate_fitness(kunangs, df)

[ 9.94141658 11.41015969 10.05510862  9.75697276]


array([[ 7.91348267,  1.49501169,  2.59715461,  4.01687221,  8.15752291,
         4.24880893,  1.02554559,  8.03379343,  8.83240293,  5.42219789,
         9.91206883,  3.16255079,  9.65081878,  1.49283316,  5.29332248,
         8.54417041,  8.54662427,  8.35472003,  9.51850189,  2.59406872,
         6.3894718 ,  3.42192787,  5.76101831,  7.72938651,  9.94141658],
       [ 4.63501508,  4.97860023,  7.5483839 ,  4.46017026,  8.76932727,
         4.04407313,  8.51815035,  5.33955442,  3.46262213,  7.08256716,
         6.6121777 ,  7.14527499,  4.75870229,  8.13551065,  6.78056696,
         8.67635263,  2.64165705,  8.93148302,  5.01506629,  5.95743055,
         4.23073229,  9.50899547,  9.92359357,  9.10162877, 11.41015969],
       [ 5.34812567,  9.76793648,  6.8310175 ,  8.53917504,  7.63431022,
         9.76316366,  1.956988  ,  4.2742454 ,  6.14971457,  1.63646774,
         7.45632586,  1.2629425 ,  7.18041886,  9.64337859,  5.14809428,
         2.24573506,  2.92470499,  1.75134764,  9

In [ ]:
d = diskritisasi(kunangs)
d

array([[ 6, 13,  1, 19,  2, 11, 21,  3,  5, 14,  9, 22, 20, 23,  0,  7,
         4, 17, 15, 16,  8, 18, 12, 10],
       [16,  8,  5, 20,  3,  0, 12,  1, 18,  7, 19, 10, 14,  9, 11,  2,
        13,  6, 15,  4, 17, 23, 21, 22],
       [11,  9, 17,  6, 15, 16, 21, 19,  7, 14,  0,  8, 23,  2, 12, 10,
         4,  3, 20, 18, 13,  5,  1, 22],
       [17, 11, 13, 16,  7,  1,  3,  2, 19, 12,  6, 18,  5, 14, 21, 23,
        15,  9, 10,  8, 22,  4, 20,  0]])

In [ ]:
calc_dist( d, df.values)

array([1.40502282, 2.3003391 , 1.92592415, 1.22389419, 1.59415495,
       1.21228992, 1.75118421, 1.22307316, 1.4667433 , 1.53275645,
       1.1873652 , 1.76146477, 1.37060519, 1.44143891, 1.78992024,
       3.4446807 , 2.23790759, 1.00943052, 2.22552855, 2.04784607,
       2.24294439, 2.57394287, 0.94889783, 1.09771246])

In [ ]:
fitness = np.array(list(map( lambda x: calc_dist( x, df.values), d )))
fitness

array([ 9.94141658, 11.41015969, 10.05510862,  9.75697276])

In [ ]:
fitness = fitness.reshape( (-1,1) )
fitness

array([[ 9.94141658],
       [11.41015969],
       [10.05510862],
       [ 9.75697276]])

In [ ]:
final = np.concatenate( (kunangs, fitness), axis=1 )
final

array([[ 7.91348267,  1.49501169,  2.59715461,  4.01687221,  8.15752291,
         4.24880893,  1.02554559,  8.03379343,  8.83240293,  5.42219789,
         9.91206883,  3.16255079,  9.65081878,  1.49283316,  5.29332248,
         8.54417041,  8.54662427,  8.35472003,  9.51850189,  2.59406872,
         6.3894718 ,  3.42192787,  5.76101831,  7.72938651,  9.94141658],
       [ 4.63501508,  4.97860023,  7.5483839 ,  4.46017026,  8.76932727,
         4.04407313,  8.51815035,  5.33955442,  3.46262213,  7.08256716,
         6.6121777 ,  7.14527499,  4.75870229,  8.13551065,  6.78056696,
         8.67635263,  2.64165705,  8.93148302,  5.01506629,  5.95743055,
         4.23073229,  9.50899547,  9.92359357,  9.10162877, 11.41015969],
       [ 5.34812567,  9.76793648,  6.8310175 ,  8.53917504,  7.63431022,
         9.76316366,  1.956988  ,  4.2742454 ,  6.14971457,  1.63646774,
         7.45632586,  1.2629425 ,  7.18041886,  9.64337859,  5.14809428,
         2.24573506,  2.92470499,  1.75134764,  9

In [ ]:
def sort_individu(kunangs_with_f):
    return kunangs_with_f[ kunangs_with_f[:,-1].argsort() ]

In [ ]:
kunangs[:,-1].argsort()

array([2, 3, 0, 1])

In [ ]:
sort_individu(kunangs)

array([[5.34812567, 9.76793648, 6.8310175 , 8.53917504, 7.63431022,
        9.76316366, 1.956988  , 4.2742454 , 6.14971457, 1.63646774,
        7.45632586, 1.2629425 , 7.18041886, 9.64337859, 5.14809428,
        2.24573506, 2.92470499, 1.75134764, 9.46049487, 3.97718096,
        9.25040331, 3.20078346, 9.9881189 , 6.55069277],
       [9.87882148, 4.42757287, 4.76075164, 4.69656794, 8.45317029,
        6.53833604, 5.79598259, 3.4779617 , 7.75106301, 7.05882705,
        7.05963511, 1.14161383, 5.46478012, 1.35899315, 6.57180777,
        6.92445239, 2.59513232, 1.07621693, 6.20097675, 4.88716133,
        8.50255648, 6.69635596, 8.27286865, 6.84416644],
       [7.91348267, 1.49501169, 2.59715461, 4.01687221, 8.15752291,
        4.24880893, 1.02554559, 8.03379343, 8.83240293, 5.42219789,
        9.91206883, 3.16255079, 9.65081878, 1.49283316, 5.29332248,
        8.54417041, 8.54662427, 8.35472003, 9.51850189, 2.59406872,
        6.3894718 , 3.42192787, 5.76101831, 7.72938651],
       [4.635